In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit

In [3]:
txn_df = pd.read_json("transactions.txt",lines = True)
txn_df.replace("",np.nan,inplace = True)

/var/folders/70/1_tl7sh93ls9_l_lcqmkctg40000gn/T/ipykernel_80420/3049029488.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  txn_df.replace("",np.nan,inplace = True)


In [4]:
pd.set_option('display.max_columns',None)
txn_df.head()

,accountNumber,customerId,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,posConditionCode,merchantCategoryCode,currentExpDate,accountOpenDate,dateOfLastAddressChange,cardCVV,enteredCVV,cardLast4Digits,transactionType,echoBuffer,currentBalance,merchantCity,merchantState,merchantZip,cardPresent,posOnPremises,recurringAuthInd,expirationDateKeyInMatch,isFraud
0,737265056,737265056,5000,5000.0,2016-08-13T14:27:32,98.55,Uber,US,US,02,01,rideshare,06/2023,2015-03-14,2015-03-14,414,414,1803,PURCHASE,NaN,0.0,NaN,NaN,NaN,False,NaN,NaN,False,False
1,737265056,737265056,5000,5000.0,2016-10-11T05:05:54,74.51,AMC #191138,US,US,09,01,entertainment,02/2024,2015-03-14,2015-03-14,486,486,767,PURCHASE,NaN,0.0,NaN,NaN,NaN,True,NaN,NaN,False,False
2,737265056,737265056,5000,5000.0,2016-11-08T09:18:39,7.47,Play Store,US,US,09,01,mobileapps,08/2025,2015-03-14,2015-03-14,486,486,767,PURCHASE,NaN,0.0,NaN,NaN,NaN,False,NaN,NaN,False,False
3,737265056,737265056,5000,5000.0,2016-12-10T02:14:50,7.47,Play Store,US,US,09,01,mobileapps,08/2025,2015-03-14,2015-03-14,486,486,767,PURCHASE,NaN,0.0,NaN,NaN,NaN,False,NaN,NaN,False,False
4,830329091,830329091,5000,5000.0,2016-03-24T21:04:46,71.18,Tim Hortons #947751,US,US,02,01,fastfood,10/2029,2015-08-06,2015-08-06,885,885,3143,PURCHASE,NaN,0.0,NaN,NaN,NaN,True,NaN,NaN,False,False


Dataset has 786,363 rows and 29 features

In [5]:
txn_df.shape

(786363, 29)

In [6]:
# Column names
txn_df.columns

Index(['accountNumber', 'customerId', 'creditLimit', 'availableMoney',
       'transactionDateTime', 'transactionAmount', 'merchantName',
       'acqCountry', 'merchantCountryCode', 'posEntryMode', 'posConditionCode',
       'merchantCategoryCode', 'currentExpDate', 'accountOpenDate',
       'dateOfLastAddressChange', 'cardCVV', 'enteredCVV', 'cardLast4Digits',
       'transactionType', 'echoBuffer', 'currentBalance', 'merchantCity',
       'merchantState', 'merchantZip', 'cardPresent', 'posOnPremises',
       'recurringAuthInd', 'expirationDateKeyInMatch', 'isFraud'],
      dtype='object')

In [7]:
txn_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 786363 entries, 0 to 786362
Data columns (total 29 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   accountNumber             786363 non-null  int64  
 1   customerId                786363 non-null  int64  
 2   creditLimit               786363 non-null  int64  
 3   availableMoney            786363 non-null  float64
 4   transactionDateTime       786363 non-null  object 
 5   transactionAmount         786363 non-null  float64
 6   merchantName              786363 non-null  object 
 7   acqCountry                781801 non-null  object 
 8   merchantCountryCode       785639 non-null  object 
 9   posEntryMode              782309 non-null  object 
 10  posConditionCode          785954 non-null  object 
 11  merchantCategoryCode      786363 non-null  object 
 12  currentExpDate            786363 non-null  object 
 13  accountOpenDate           786363 non-null  o

Datatype and % of missing values in each column

In [8]:
def sniff(df):
  """
  Returns sample data, data type and percentage missing for each column of a data frame

  Parameter and type: 
    df(Pandas Dataframe): Dataframe

  Return:
    :rtype: Dataframe
    :return: Columns mentioned in rows with sample, data type and percent missing as columns
  """
  with pd.option_context("display.max_colwidth", 20):
    info = pd.DataFrame()
    info['sample'] = df.iloc[0]
    info['data type'] = df.dtypes
    info['percent missing'] = df.isnull().sum()*100/len(df)
    return info.sort_values('data type')

In [9]:
sniff(txn_df)

,sample,data type,percent missing
isFraud,False,bool,0.000000
cardPresent,False,bool,0.000000
expirationDateKeyInMatch,False,bool,0.000000
cardLast4Digits,1803,int64,0.000000
enteredCVV,414,int64,0.000000
cardCVV,414,int64,0.000000
accountNumber,737265056,int64,0.000000
creditLimit,5000,int64,0.000000
customerId,737265056,int64,0.000000
recurringAuthInd,NaN,float64,100.000000


Dropping all null columns


In [10]:
col_to_drop = ['recurringAuthInd','posOnPremises','merchantZip','merchantState','merchantCity','echoBuffer']
txn_df.drop(col_to_drop,axis = 1,inplace = True)

### Change datatype of columns




In [11]:
def change_dtype(df,columns,new_dtype):
  """
  Function converting datatype of specified columns to a new datatype 

  Parameter and type: 
    df(Pandas Dataframe): Dataframe

  Return:
    :rtype: Dataframe
    :return: Dataframe with new columns (changed data type)
  """
  for col in columns :
    df[col] = df[col].astype(new_dtype)


Numeric -> Object


In [12]:
num_cols = ['cardLast4Digits','enteredCVV','cardCVV','accountNumber','customerId']
change_dtype(txn_df, num_cols, 'object')

Object -> Datetime

In [13]:
object_cols = ['transactionDateTime','accountOpenDate','currentExpDate','dateOfLastAddressChange']
change_dtype(txn_df, object_cols, 'datetime64[ns]')

Train and test set split (80:20)


In [14]:
# Proportion of positive labels before splitting
txn_df.isFraud.value_counts()/len(txn_df)

isFraud
False    0.98421
True     0.01579
Name: count, dtype: float64

In [15]:
# Stratified split to keep the distribution of target variable same
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(txn_df, txn_df["isFraud"]):
    strat_train_set = txn_df.loc[train_index]
    strat_test_set = txn_df.loc[test_index]

Saving the data set to pickle


In [16]:
pd.to_pickle(txn_df, 'transaction_dataset', protocol=4)
pd.to_pickle(strat_train_set, 'train_df', protocol=4)
pd.to_pickle(strat_test_set, 'test_df', protocol=4)